# Make a fake stead-state landscape using SPACE + Run Hyland for creating landslides.

* Campforts, B., Shobe, C. M., Overeem, I., & Tucker, G. E. (2022). The Art of Landslides: How Stochastic Mass Wasting Shapes Topography and Influences Landscape Dynamics. Journal of Geophysical Research: Earth Surface, 127(8), 1–16. [doi: 10.1029/2022JF006745](https://doi.org/10.1029/2022JF006745)
* Campforts B., Shobe C.M., Steer P., Vanmaercke M., Lague D., Braun J. (2020) HyLands 1.0: a hybrid landscape evolution model to simulate the impact of landslides and landslide-derived sediment on landscape evolution. Geosci Model Dev: 13(9):3863–86, [doi: 10.5194/gmd-13-3863-2020](https://doi.org/10.5194/gmd-13-3863-2020).

### Step 1: Import the necessary libraries 

The BedrockLandslider and SPACE components are required, as are the model grid component and a flow routing component. We will use the `PriorityFloodFlowRouter` that takes care of routing flow across flats or pits in a digital elevation model.

In [11]:
import copy

import matplotlib as mpl
import matplotlib.pyplot as plt  # For plotting results; optional
import numpy as np

from landlab import RasterModelGrid  # Grid utility
from landlab import imshow_grid, imshowhs_grid  # For plotting results; optional
from landlab.components import BedrockLandslider  # BedrockLandslider model
from landlab.components import SpaceLargeScaleEroder  # SPACE model
from landlab.components import PriorityFloodFlowRouter
from landlab.io.esri_ascii import write_esri_ascii

### Step 2: Define the model domain and initial conditions

In [12]:
# Set grid parameters
num_rows = 30
num_columns = 30
node_spacing = 25.0

# Create initial model topography:

# Instantiate model grid
mg = RasterModelGrid((num_rows, num_columns), node_spacing) 
mg.add_zeros("node", "topographic__elevation") # add field ’topographic elevation’ to the grid
# add topographic roughness
random_noise = (np.random.rand(len(mg.node_y)) / 1000.0)  # impose topography values on model grid
mg["node"]["topographic__elevation"] += random_noise

mg.add_zeros("node", "soil__depth") # Create a field 'soil__depth' to the grid
mg.at_node["soil__depth"][mg.core_nodes] = 1.0  #Set s m of initial soil depth at core nodes

# Add field 'bedrock__elevation' to the grid
mg.add_zeros("bedrock__elevation", at="node")

# Yield 'topographic elevation' by summing 'soil__depth' and 'bedrock__elevation'
mg.at_node["bedrock__elevation"][:] = mg.at_node["topographic__elevation"]
mg.at_node["topographic__elevation"][:] += mg.at_node["soil__depth"]

### Step 3: Set the boundary conditions

The user must determine the boundary conditions of the model domain (i.e., determine across which boundaries water and sediment may flow). Boundary conditions are controlled by setting the status of individual nodes or grid edges (see Hobley et al., 2017). Here, we keep all boundaries open so that water and sediment can leave the domain at all times over all edges. 

In [13]:
# Open all model boundary edges
mg.set_closed_boundaries_at_grid_edges(
    bottom_is_closed=True,
    left_is_closed=True,
    right_is_closed=True,
    top_is_closed=True,
)
mg.status_at_node[5] = 1 

### Step 4: Initialize the flow director and SPACE 

Like most Landlab components, PriorityFloodFlowRouter, SPACE and BedrockLandslider are written as a Python classes. The class was imported at the beginning of the driver script (step 1). In this step, the user declares the instance of the PriorityFloodFlowRouter and SPACE classes and sets any relevant model parameters. At this point we do not yet make an instance of the BedrockLandslider component. 

In [14]:
# Instantiate flow router
fr = PriorityFloodFlowRouter(mg, flow_metric="D8", suppress_out=True)

# Instantiate SPACE model with chosen parameters
sp = SpaceLargeScaleEroder(
    mg,
    K_sed=2.5e-5,
    K_br=2.5e-5,
    F_f=0.0,
    phi=0.0,
    H_star=1.0,
    v_s=1,
    m_sp=0.5,
    n_sp=1.0,
    sp_crit_sed=0,
    sp_crit_br=0,
)

### Step 5: Run the time loop to develop a landscape without landslide activity

The SPACE component calculates sediment entrainment and deposition, bedrock erosion, and changes in land surface elevation over time. The code shown below is an example of how to run the SPACE model over several model timesteps. In the example below, SPACE is run in a loop that executes until elapsed model time has reached a user-defined run time. The user is also responsible for choosing the model timestep. Within the loop, the following steps occur:

1. The flow router runs first to determine topographic slopes and water discharge at all nodes on the model domain.
2. The depression finder and router runs to map any nodes located in local topographic minima (i.e., nodes that water cannot drain out of) and to establish flow paths across the surface of these “lakes.” Using the depression finder and router is optional. However, because the SPACE model may in certain situations create local minima, using the depression finder and router can prevent the development of fatal instabilities.
3. The depression finder and router generates a list of flooded nodes, which is then saved as a variable called “flooded” and passed to the SPACE model.
4. The SPACE model runs for the duration of a single timestep, computing sediment transport, bedrock erosion, and topographic surface evolution.
5. The elapsed time is updated.

In [15]:
# Set model timestep
timestep = 5e2  # years

# Set elapsed time to zero
elapsed_time = 0.0  # years

# Set timestep count to zero
count = 0

# Set model run time
run_time = 5e5  # years

# Array to save sediment flux values
sed_flux = np.zeros(int(run_time // timestep))

# Uplift rate in m/yr
U = 1e-2

cmap = copy.copy(mpl.colormaps["terrain"])

while elapsed_time < run_time:  # time units of years
    # Insert uplift at core nodes
    mg.at_node["bedrock__elevation"][mg.core_nodes] += U * timestep
    mg.at_node["topographic__elevation"][:] = (
        mg.at_node["bedrock__elevation"] + mg.at_node["soil__depth"]
    )

    # Run the flow router
    fr.run_one_step()

    # Run SPACE for one time step
    sp.run_one_step(dt=timestep)

    # Add to value of elapsed time
    elapsed_time += timestep

    if np.mod(elapsed_time, 1e5) == 0:
        print("%.2f of model run completed" % (elapsed_time / run_time))

0.20 of model run completed
0.40 of model run completed
0.60 of model run completed
0.80 of model run completed
1.00 of model run completed


#### **Using `pickle` to export & import the landlab grid object **

In [16]:
#See available fields in the landlab grid object
mg.fields()

{'at_node:bedrock__elevation',
 'at_node:depression_free_elevation',
 'at_node:drainage_area',
 'at_node:flood_status_code',
 'at_node:flow__link_to_receiver_node',
 'at_node:flow__receiver_node',
 'at_node:flow__receiver_proportions',
 'at_node:flow__upstream_node_order',
 'at_node:sediment__flux',
 'at_node:sediment__influx',
 'at_node:sediment__outflux',
 'at_node:soil__depth',
 'at_node:squared_length_adjacent',
 'at_node:surface_water__discharge',
 'at_node:topographic__elevation',
 'at_node:topographic__steepest_slope',
 'at_node:water__unit_flux_in'}

In [17]:
import pickle

# Exporting landlab grid object
with open('pre-slide_fakelandscape.pickle', 'wb') as f:
    pickle.dump(mg, f)

In [18]:
mg.fields()

{'at_node:bedrock__elevation',
 'at_node:depression_free_elevation',
 'at_node:drainage_area',
 'at_node:flood_status_code',
 'at_node:flow__link_to_receiver_node',
 'at_node:flow__receiver_node',
 'at_node:flow__receiver_proportions',
 'at_node:flow__upstream_node_order',
 'at_node:sediment__flux',
 'at_node:sediment__influx',
 'at_node:sediment__outflux',
 'at_node:soil__depth',
 'at_node:squared_length_adjacent',
 'at_node:surface_water__discharge',
 'at_node:topographic__elevation',
 'at_node:topographic__steepest_slope',
 'at_node:water__unit_flux_in'}

## Visualization of results

### Step 6: Initialize the flow director and SPACE and BedrockLandslider components

In [19]:
# Instantiate flow router, with additional multiple flow director for hillslopes
fr = PriorityFloodFlowRouter(
    mg,
    flow_metric="D8",
    separate_hill_flow=True,
    hill_flow_metric="Quinn",
    update_hill_flow_instantaneous=True,
)

# Instantiate SPACE model with chosen parameters
hy = BedrockLandslider(
    mg,
    angle_int_frict=0.4,
    cohesion_eff=1e3,
    landslides_return_time=1000,
    landslides_on_boundary_nodes=False,
)


### Step 7: Run the time loop over 200 years to develop a landscape with landslide activity

Like before, uplift will be inserted, the flow routers will be updated and the SAPCELargeScaleEroder will be run, but this time, landslides will be actively simulated.  

In [20]:
# Reset elevation back to elevation simulated without landslides to test various landslide configuration settings
z_before_LS = np.array(mg["node"]["topographic__elevation"])

mg["node"]["topographic__elevation"][:] = z_before_LS
timestep = 2  # years
landslides_size_all_steps = []

for i in range(10):
    # Insert uplift at core nodes
    mg.at_node["bedrock__elevation"][mg.core_nodes] += U * timestep
    mg.at_node["topographic__elevation"][:] = (
        mg.at_node["bedrock__elevation"] + mg.at_node["soil__depth"]
    )

    # Run the flow router
    fr.run_one_step()

    # Run SPACE for one time step
    sp.run_one_step(dt=timestep)

    # Run BedrockLandslider for one time step
    hy.run_one_step(dt=timestep)

    # Store landslide sizes of current time step into general ls_size list
    landslides_size_all_steps = np.append(landslides_size_all_steps, hy.landslides_size)
    

### Exporting post-slide landscpae

In [21]:
# Exporting landlab grid object
with open('post-slide_fakelandscape.pickle', 'wb') as f:
    pickle.dump(mg, f)

In [22]:
mg.fields()a

{'at_node:LS_sediment__flux',
 'at_node:bedrock__elevation',
 'at_node:depression_free_elevation',
 'at_node:drainage_area',
 'at_node:flood_status_code',
 'at_node:flow__link_to_receiver_node',
 'at_node:flow__receiver_node',
 'at_node:flow__receiver_proportions',
 'at_node:flow__upstream_node_order',
 'at_node:hill_flow__receiver_node',
 'at_node:hill_flow__receiver_proportions',
 'at_node:hill_flow__upstream_node_order',
 'at_node:hill_topographic__steepest_slope',
 'at_node:landslide__deposition',
 'at_node:landslide__erosion',
 'at_node:landslide_sediment_point_source',
 'at_node:sediment__flux',
 'at_node:sediment__influx',
 'at_node:sediment__outflux',
 'at_node:soil__depth',
 'at_node:squared_length_adjacent',
 'at_node:surface_water__discharge',
 'at_node:topographic__elevation',
 'at_node:topographic__steepest_slope',
 'at_node:water__unit_flux_in'}

### Click here for more <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>